In [6]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [1]:
committees = ScrapedCommittee.objects.filter(
    Q(name__icontains='sound transit') | Q(name__icontains='mass transit'))
ballots = SOSPDCMap.objects.filter(pdc_filer_id__in=[c.filer_id for c in committees])
for ballot in ballots:
    print ballot.sos_ballot_id, ballot.pdc_filer_id

for ballot in BallotRaceSummary.objects.filter(race_id__gte=999990):
    print ballot

contribs = ScrapedContribution.objects.filter(filing_committee__in=committees).order_by('-amount')


refunds = ScrapedRefund.objects.filter(filing_committee__in=committees)

NameError: name 'ScrapedCommittee' is not defined

In [2]:
st_district = BallotDistrict.objects.get(district_id=settings.SOUND_TRANSIT_DISTRICT_ID)
election = BallotElection.objects.get(is_current=True)

mtn = BallotRaceSummary.objects.crate(
    ballot_id=999991,
    ballot_name='Yes',
    ballot_name_with_party='Yes',
    race_id=999999,
    race_name='Proposition 1 (Sound Transit 3) - Light rail, commuter rail, and bus service expansion, SOUND TRANSIT'
)

nost3 = BallotRaceSummary.objects.crate(
    ballot_id=999992,
    ballot_name='No',
    ballot_name_with_party='No',
    race_id=999999,
    race_name='Proposition 1 (Sound Transit 3) - Light rail, commuter rail, and bus service expansion, SOUND TRANSIT'
)

NameError: name 'BallotDistrict' is not defined

In [3]:
# SEIU historical contribution processing to fix spreadsheet error

contribs = PDCContribution.objects.exclude(is_superseded=True).filter(
    Q(donor__full_name__icontains='seiu') | Q(donor__full_name__icontains='service employee'))

groups = {
    'house': {},
    'senate': {}
}

for contrib in contribs:
    cmte = contrib.filer_committee
    cmte_year = PDCCommitteeYear.objects.filter(
        committee=cmte, year__in=[contrib.rcpt_date.year, contrib.rcpt_date.year + 1])
    key = ''
    for year in cmte_year:
        if year.jurisdiction.find('LEG DISTRICT') != -1:
            if year.jurisdiction.find('HOUSE') != -1:
                key = 'house'
            else:
                key = 'senate'

    if key:
        if contrib.rcpt_date.year not in groups[key]:
            groups[key][contrib.rcpt_date.year] = 0.
        groups[key][contrib.rcpt_date.year] += float(contrib.amount)

NameError: name 'PDCContribution' is not defined

In [4]:
for year in range(1996, 2017):
    print year
    for chamber in ['senate']:
        if year in groups[chamber]:
            print chamber, groups[chamber][year]
        else:
            print 'No data'

1996


NameError: name 'groups' is not defined

In [5]:
def summer(amount, iters):
    if amount > 1000000000000:
        print 'Took %d iters to reach %d' % (iters, amount)
    else:
        iters += 1
        print iters
        return summer(amount * 2, iters)

summer(100, )

TypeError: summer() takes exactly 2 arguments (1 given)